In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/firewall-malware-analysis/df_minimization.csv')

In [ ]:
exception_col = 'Label'

for col in df.columns:
  if col != exception_col:
    df[col] = df[col].astype('float32')

df[exception_col] = df[exception_col].astype("int8")



In [ ]:
DROP_COLS = [
    "Flow ID",
    "Source IP",
    "Source Port",
    "Destination IP",
    "Destination Port",
    "Timestamp",
    "Fwd Header Length.1"
]

df.drop(columns=DROP_COLS, inplace=True, errors="ignore")


In [ ]:
X = df.drop("Label", axis=1).values
y = df["Label"].values

X = X.reshape(X.shape[0], X.shape[1], 1)


In [ ]:
import numpy as np
print("The Shape of the Input data : " + str(X.shape))
print("The Shape of the Output data : " + str(y.shape))

The Shape of the Input data : (50000, 76, 1)
The Shape of the Output data : (50000,)


In [ ]:
class Relu:
  def forward(self, input_data):
    return np.maximum(0, input_data)

  def backward(self, grad_out):
        return grad_out * self.mask

In [ ]:
class Conv1DLayer:
    def __init__(self, num_filters, kernel_size):
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.filters = None

    def forward(self, x):
        batch, time, features = x.shape

        if self.filters is None:
            self.filters = np.random.randn(
                self.num_filters, self.kernel_size, features
            ) * 0.01

        output_time = time - self.kernel_size + 1
        out = np.zeros((batch, output_time, self.num_filters))

        for b in range(batch):
            for f in range(self.num_filters):
                for t in range(output_time):
                    window = x[b, t:t+self.kernel_size, :]
                    out[b, t, f] = np.sum(window * self.filters[f])

        return out


In [ ]:
class MaxPooling1DLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, x):
        batch, time, features = x.shape
        output_time = time // self.pool_size

        out = np.zeros((batch, output_time, features))

        for b in range(batch):
            for t in range(output_time):
                window = x[b,
                           t*self.pool_size:(t+1)*self.pool_size,
                           :]
                out[b, t, :] = np.max(window, axis=0)

        return out


In [ ]:
import numpy as np

class Dense:
    def __init__(self, input_dim, output_dim, activation=None):
        self.W = np.random.randn(input_dim, output_dim) * 0.01
        self.b = np.zeros((1, output_dim))
        self.activation = activation

    def forward(self, x):
        self.x = x
        z = np.dot(x, self.W) + self.b

        if self.activation == "relu":
            self.out = np.maximum(0, z)

        elif self.activation == "softmax":
            exp = np.exp(z - np.max(z, axis=1, keepdims=True))
            self.out = exp / np.sum(exp, axis=1, keepdims=True)

        elif self.activation == "sigmoid":
            self.out = 1 / (1 + np.exp(-z))

        else:
            self.out = z

        return self.out



In [ ]:
conv1 = Conv1DLayer(num_filters=8, kernel_size=3)
x = conv1.forward(X)

pool1 = MaxPooling1DLayer(pool_size=2)
x = pool1.forward(x)

print("After first pooling:", x.shape)



conv2 = Conv1DLayer(num_filters=16, kernel_size=3)
x = conv2.forward(x)

pool2 = MaxPooling1DLayer(pool_size=2)
x = pool2.forward(x)

print("After second pooling:", x.shape)

batch, time, features = x.shape
x = x.reshape(batch, time * features)

dense = Dense(input_dim=time * features, output_dim=2, activation="softmax")
predictions = dense.forward(x)


After first pooling: (50000, 37, 8)
After second pooling: (50000, 17, 16)


In [ ]:
predictions

NameError: name 'predictions' is not defined